In [ ]:
import cv2
import urllib.request
from PIL import Image
from io import BytesIO

import numpy as np
import pandas as pd
from scipy.linalg import sqrtm
from scipy.signal import convolve2d

from tqdm.auto import tqdm

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
def mk_matrix(img):
    ''' Convert PIL [img] into RGB and HSV Matrix,
    and put them into the [img] Object.
    '''
    img.matrix = dict()
        
    rgb = np.array(img.convert('RGB'))
    
    hsv = cv2.cvtColor(rgb, cv2.COLOR_RGB2HSV)
    
    img.matrix['RGB'] = rgb
    img.matrix['HSV'] = hsv
    
    return img

In [ ]:
def ravel(d, full=False, num=500):
    ''' Ravel the 1st and the 2nd Dimensions,
    only keep 500 Pixels to save Computation Time. 
    '''
    s = d.shape
    n = s[0] * s[1]
    d = d.reshape((n, s[2]))
    if n > num and not full:
        return d[range(0, n, int(n/num))]
    else:
        return d

def convert(rgb):
    ''' Convert RGB [rgb] values into #FFFFFF like Color String'''
    
    def n2s(n):
        s = hex(n).replace('x', '')
        return s[-2:]
    
    return '#' + ''.join([n2s(n) for n in rgb])

In [ ]:
def color_space(im):
    ''' Draw Color Space of Image [im] '''
    mk_matrix(im)

    v_rgb = ravel(im.matrix['RGB'])
    df = pd.DataFrame(ravel(im.matrix['RGB']), columns=['R', 'G', 'B'])
    df[['H', 'S', 'V']] = ravel(im.matrix['HSV'])
    df['color'] = [convert(e) for e in v_rgb]

    subplot_titles = ('HS', 'SV', 'HV')
    subplot_pos = ((1, 1), (1, 2), (2, 1))
    fig = make_subplots(rows=2, cols=2, subplot_titles=subplot_titles)

    for pos, title in zip(subplot_pos, subplot_titles):
        x, y = title
        row, col = pos

        _fig = px.scatter(df, x=x, y=y, title=title)
        _fig.data[0]['marker']['color'] = df['color']

        kwargs = dict(
            row=row,
            col=col
        )

        fig.add_trace(
            _fig.data[0],
            **kwargs
        )

        fig.update_xaxes(title_text=x, **kwargs)
        fig.update_yaxes(title_text=y, **kwargs)
        
    fig1 = px.imshow(im)
    fig.add_trace(
        fig1.data[0],
        row=2, col=2
    )

    fig.update_layout(dict(
        height=800,
        width=900,
        title=url
    ))
    fig.show()


#     fig1.show()

In [ ]:
# Plot Converted Img
def plot_hsv(hsv):
    ''' Build Img by [hsv] matrix,
    and Plot the ColorSpace.
    '''
    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    im = Image.fromarray(rgb)
    color_space(im)
    return im

In [ ]:
url = 'http://localhost:8000/yosemite-5932x3337-5k-4k-wallpaper-8k-forest-osx-apple-mountains-181.jpg'
url = 'http://localhost:8000/macOS-Sierra-Wallpaper-Macbook-Wallpaper.jpg'
url = 'http://localhost:8000/DT86.jpg'
url = 'http://localhost:8000/132457451652521207_new.jpg'
url = 'http://localhost:8000/R18faf19c0f6ad585eea167b79e4fb065.jfif'
req = urllib.request.urlopen(url)
imt = Image.open(BytesIO(req.read()))
color_space(imt)

In [ ]:
# Converted Image
url = 'http://localhost:8000/R18faf19c0f6ad585eea167b79e4fb065.jfif'
url = 'http://localhost:8000/132457451652521207_new.jpg'
req = urllib.request.urlopen(url)
im = Image.open(BytesIO(req.read()))
color_space(im)

In [ ]:
# Get Target Distribution
hsvt = imt.matrix['HSV'].copy()[::5, ::5, :]
hsvt_ravel = ravel(hsvt, num=2000)
hsvt_ravel.shape

In [ ]:
# Get Source Distribution
hsv = im.matrix['HSV'].copy()[::2, ::2, :]
hsv_ravel = ravel(hsv, full=True)
hsv_ravel.shape

In [ ]:
# Match using Nearest Matching
dist = []
for j in tqdm(range(len(hsvt_ravel))):
    dist.append(np.linalg.norm(hsv_ravel - hsvt_ravel[j], axis=1))

dist = np.array(dist)
dist.shape

idxs = np.argmax(dist, axis=0)
idxs.shape

In [ ]:
# Convert the HSV using the Nearest Matching Method
new_hsv = hsv_ravel.copy()
for j in tqdm(range(len(idxs))):
    new_hsv[j] = hsvt_ravel[idxs[j]]
    
new_hsv_2d = new_hsv.reshape(hsv.shape)

h = np.ones((3, 3)) / (3 * 3)
for j in [1, 2]:
    new_hsv_2d[:, :, j] = convolve2d(new_hsv_2d[:, :, j], h, mode='same')

new_hsv_2d.shape

In [ ]:
# Plot the Converted Img
color_space(im)
new_im = plot_hsv(new_hsv_2d)
color_space(imt)

In [ ]:
hsv = im.matrix['HSV'].copy()
print(hsv.shape)

# Convert V
hsv[:, :, 2] = hsv[:, :, 2] / 255 * 200
hsv[:, :, 2][hsv[:, :, 0] > 60] = hsv[:, :, 2][hsv[:, :, 0] > 60] / 200 * 150
hsv[:, :, 2][hsv[:, :, 0] <= 60] = hsv[:, :, 2][hsv[:, :, 0] <= 60] / 200 * 120 + 120

# Convert S
hsv[:, :, 1] = hsv[:, :, 1] * (1 - hsv[:, :, 2] / 255)
hsv[:, :, 1] = hsv[:, :, 1] / 255 * 150
hsv[:, :, 1][hsv[:, :, 2] > 155] = hsv[:, :, 1][hsv[:, :, 2] > 155] / 255 * 100

# Convert H
# hsv[:, :, 0][hsv[:, :, 0] > 60] = ((hsv[:, :, 0][hsv[:, :, 0] > 60] - 100) * 2 + 100) % 180

In [ ]:
new_im = plot_hsv(hsv)

In [ ]:
new_im.save('a.png')